# Spam 메일 분류

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving spam1.csv to spam1 (1).csv


In [ ]:
df = pd.read_csv(filename)
df.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


In [ ]:
df.shape

(5572, 5)

In [ ]:
df['v1,v2,,,'][0].strip(', ')[4:]

'"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."'

In [ ]:
del df['Unnamed: 2']
del df['Unnamed: 3']
del df['Unnamed: 4']
df['label'] = df.v1.apply(lambda x : x for x == ham)
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## 전처리

In [ ]:
df.v2.unique()

In [ ]:
df.drop_duplicates('v2', keep='first', inplace=True)

In [ ]:
import re

df['content'] = df.v2.apply(lambda x: re.sub('[^A-Za-z]', ' ', x))

In [ ]:
# 소문자 변환

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.v2, df.v1, test_size=0.2,         # 20000개 - 훈련용 , 5000개 - 테스트용
    stratify=df.v1, random_state=2021        # Stratify : 균등하게 분배
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4457,), (1115,), (4457,), (1115,))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tvect1 = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tvect1.fit(X_train)
X_train_tv1 = tvect1.transform(X_train)
X_test_tv1 = tvect1.transform(X_test)
X_train_tv1.shape, X_test_tv1.shape

((4457, 29219), (1115, 29219))

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# 객체 생성 및 훈련 - 훈련 데이터의 X와 y 모두 필요
# CountVectorizer, ngram_range = (1,1)
lr = LogisticRegression(max_iter=500)
%time lr.fit(X_train_tv1, y_train)

CPU times: user 155 ms, sys: 131 ms, total: 286 ms
Wall time: 163 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
pred = lr.predict(X_test_tv1)

In [ ]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, pred)
score

0.9515695067264573